This Jupyter Notebook is used for exploring different problems, and attempts at solving them

# groupby -> resample -> agg

Examining the strange result of using `resample` + `agg` on `groupby`,
stubled upon in [`02-contributors_graph.py`](./02-contributors_graph.py)

In [1]:
import json

import pandas as pd

In [2]:
with open('../../data/examples/stats/qtile.timeline.purpose-to-type.json', mode='r') as json_fp:
    data = json.load(json_fp)

In [3]:
data.keys()

dict_keys(['qtile'])

In [4]:
df = pd.DataFrame.from_records(data['qtile'])
df.shape

(5350, 97)

In [5]:
df = df[df['n_parents'] == 1]
df.shape

(5347, 97)

In [6]:
df = df.dropna(subset=['author.timestamp', 'committer.timestamp'], how='any')
df.shape

(5347, 97)

In [7]:
df = df.assign(
            n_commits =  1,
            author_date    = lambda x: pd.to_datetime(x['author.timestamp'],    unit='s', utc=True),
            committer_date = lambda x: pd.to_datetime(x['committer.timestamp'], unit='s', utc=True),
        )
df.shape

(5347, 100)

In [8]:
df.head(3)

,bug_id,patch_id,file_names,language:Python,type:programming,purpose:programming,+:count,+:type.code,+:purpose.programming,+:type.documentation,...,language:JSON,language:TOML,language:Git Revision List,language:SVG,language:desktop,language:CSS,language:Nix,n_commits,author_date,committer_date
0,all_authors-no_merges,47474a8375ae785b91992355be1678565eba9d23.v2.json,1.0,1.0,1.0,1.0,34.0,30.0,34.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2011-04-11 21:24:27+00:00,2011-04-11 21:24:27+00:00
1,all_authors-no_merges,88197e8de01d79af7858d60acb9bd57bc06b5e73.v2.json,1.0,1.0,1.0,1.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2015-10-20 11:18:55+00:00,2015-10-28 19:36:58+00:00
2,all_authors-no_merges,9f411bf9bcf74c7e28d7eaa0d1fd03c382458ba4.v2.json,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2016-08-12 04:39:33+00:00,2016-08-12 04:39:33+00:00


In [9]:
df.columns

Index(['bug_id', 'patch_id', 'file_names', 'language:Python',
       'type:programming', 'purpose:programming', '+:count', '+:type.code',
       '+:purpose.programming', '+:type.documentation', '-:count',
       '-:type.code', '-:purpose.programming', 'diff.n_files',
       'diff.hunk_span_src', 'diff.hunk_span_dst', 'diff.n_hunks',
       'diff.n_lines_added', 'diff.n_lines_removed', 'diff.n_lines_all',
       'diff.n_rem', 'diff.n_mod', 'diff.n_groups', 'diff.patch_size',
       'diff.n_add', 'diff.hunk_spread_src', 'diff.hunk_spread_dst',
       'diff.groups_spread', 'author.timestamp', 'author.tz_info',
       'author.name', 'author.email', 'committer.timestamp',
       'committer.tz_info', 'committer.name', 'committer.email', 'n_parents',
       '-:type.documentation', 'diff.spread_inner', 'purpose:test',
       '+:type.test', '+:purpose.test', '-:type.test', '-:purpose.test',
       'language:Text', 'type:prose', 'purpose:documentation',
       '+:purpose.documentation', 'languag

In [10]:
df_x = df[['author.email', 'author_date', 'n_commits', '+:count', '-:count']]
df_x

,author.email,author_date,n_commits,+:count,-:count
0,pc@gafol.net,2011-04-11 21:24:27+00:00,1,34.0,4.0
1,nxnfufunezn@gmail.com,2015-10-20 11:18:55+00:00,1,4.0,1.0
2,frostidaho@users.noreply.github.com,2016-08-12 04:39:33+00:00,1,1.0,1.0
3,dev@dariogiovannetti.net,2017-03-18 19:22:58+00:00,1,5.0,2.0
4,mcol@posteo.net,2020-06-10 18:14:23+00:00,1,2.0,1.0
...,...,...,...,...,...
5345,jdulaney@fedoraproject.org,2016-02-02 18:11:46+00:00,1,1.0,1.0
5346,uberj@onid.orst.edu,2012-05-09 06:47:52+00:00,1,1.0,1.0
5347,tycho@tycho.ws,2014-10-08 18:38:30+00:00,1,3.0,3.0
5348,sean.v.775@gmail.com,2015-01-30 01:10:39+00:00,1,52.0,73.0


## Resample only

In [11]:
resample_rate='QE'

In [12]:
agg_func_map={'n_commits': 'sum', '+:count': 'sum', '-:count': 'sum'}
agg_func_map

{'n_commits': 'sum', '+:count': 'sum', '-:count': 'sum'}

In [13]:
columns_agg = list(agg_func_map.keys())
columns_agg

['n_commits', '+:count', '-:count']

In [14]:
df_r = df_x.resample(
        resample_rate,
        on='author_date'
    )[columns_agg].agg(
        agg_func_map,
        numeric_only=True
    )
df_r.shape

(66, 3)

In [15]:
df_r

,n_commits,+:count,-:count
author_date,,,
2008-09-30 00:00:00+00:00,207,8111.0,3487.0
2008-12-31 00:00:00+00:00,73,3730.0,2403.0
2009-03-31 00:00:00+00:00,164,3330.0,1181.0
2009-06-30 00:00:00+00:00,1,1.0,2.0
2009-09-30 00:00:00+00:00,6,323.0,632.0
...,...,...,...
2023-12-31 00:00:00+00:00,107,2004.0,1202.0
2024-03-31 00:00:00+00:00,66,2385.0,540.0
2024-06-30 00:00:00+00:00,128,5381.0,1753.0


## Groupby, then resample

In [16]:
df_g = df_x.groupby('author.email')
df_g

In [17]:
df_g[columns_agg].agg(
        agg_func_map,
        numeric_only=True
    )

,n_commits,+:count,-:count
author.email,,,
0002e7@gmail.com,1,25.0,13.0
1024369+holocronweaver@users.noreply.github.com,1,125.0,2.0
104843199+jlcoulter@users.noreply.github.com,2,2.0,2.0
107062289+shyguyCreate@users.noreply.github.com,10,690.0,81.0
110528300+c0rydoras@users.noreply.github.com,1,27.0,12.0
...,...,...,...
yonnji@kitsune.one,1,82.0,0.0
yurilxc@gmail.com,1,13.0,4.0
zaheen.jamil@gmail.com,2,3.0,1.0


In [18]:
df_x.groupby('author.email').resample(
        resample_rate,
        on='author_date'
    )[columns_agg].agg(
        agg_func_map,
        numeric_only=True
    )

n_commits  \
                                                                          n_commits   
author.email                                    author_date                           
0002e7@gmail.com                                2023-09-30 00:00:00+00:00         1   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00         1   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00         2   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00         5   
                                                2024-09-30 00:00:00+00:00         3   
...                                                                             ...   
zordsdavini@gmail.com                           2021-03-31 00:00:00+00:00         1   
                                                2021-06-30 00:00:00+00:00         0   
                                                2021-09-30 00:00:00+00:00         0   
                                                2021-12-31 00:00:00+00:00         1   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00         1   

                                                                                   \
                                                                          +:count   
author.email                                    author_date                         
0002e7@gmail.com                                2023-09-30 00:00:00+00:00    25.0   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00   125.0   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00     2.0   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00    93.0   
                                                2024-09-30 00:00:00+00:00   563.0   
...                                                                           ...   
zordsdavini@gmail.com                           2021-03-31 00:00:00+00:00     1.0   
                                                2021-06-30 00:00:00+00:00     0.0   
                                                2021-09-30 00:00:00+00:00     0.0   
                                                2021-12-31 00:00:00+00:00    14.0   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00     2.0   

                                                                                   \
                                                                          -:count   
author.email                                    author_date                         
0002e7@gmail.com                                2023-09-30 00:00:00+00:00    13.0   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00     2.0   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00     2.0   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00    41.0   
                                                2024-09-30 00:00:00+00:00    25.0   
...                                                                           ...   
zordsdavini@gmail.com                           2021-03-31 00:00:00+00:00     1.0   
                                                2021-06-30 00:00:00+00:00     0.0   
                                                2021-09-30 00:00:00+00:00     0.0   
                                                2021-12-31 00:00:00+00:00     0.0   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00     1.0   

                                                                            +:count  \
                                                                          n_commits   
author.email                                    author_date                           
0002e7@gmail.com                                2023-09-30 00:00:00+00:00         1   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00         1   
104843199+jlcoulter@users.noreply.gith

In [19]:
df_g_r = df_g.resample(
        resample_rate,
        on='author_date'
    )[columns_agg].agg(
        agg_func_map,
        numeric_only=True
    )
df_g_r.shape

(1165, 9)

In [20]:
df_g_r

n_commits  \
                                                                          n_commits   
author.email                                    author_date                           
0002e7@gmail.com                                2023-09-30 00:00:00+00:00         1   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00         1   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00         2   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00         5   
                                                2024-09-30 00:00:00+00:00         3   
...                                                                             ...   
zordsdavini@gmail.com                           2021-03-31 00:00:00+00:00         1   
                                                2021-06-30 00:00:00+00:00         0   
                                                2021-09-30 00:00:00+00:00         0   
                                                2021-12-31 00:00:00+00:00         1   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00         1   

                                                                                   \
                                                                          +:count   
author.email                                    author_date                         
0002e7@gmail.com                                2023-09-30 00:00:00+00:00    25.0   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00   125.0   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00     2.0   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00    93.0   
                                                2024-09-30 00:00:00+00:00   563.0   
...                                                                           ...   
zordsdavini@gmail.com                           2021-03-31 00:00:00+00:00     1.0   
                                                2021-06-30 00:00:00+00:00     0.0   
                                                2021-09-30 00:00:00+00:00     0.0   
                                                2021-12-31 00:00:00+00:00    14.0   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00     2.0   

                                                                                   \
                                                                          -:count   
author.email                                    author_date                         
0002e7@gmail.com                                2023-09-30 00:00:00+00:00    13.0   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00     2.0   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00     2.0   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00    41.0   
                                                2024-09-30 00:00:00+00:00    25.0   
...                                                                           ...   
zordsdavini@gmail.com                           2021-03-31 00:00:00+00:00     1.0   
                                                2021-06-30 00:00:00+00:00     0.0   
                                                2021-09-30 00:00:00+00:00     0.0   
                                                2021-12-31 00:00:00+00:00     0.0   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00     1.0   

                                                                            +:count  \
                                                                          n_commits   
author.email                                    author_date                           
0002e7@gmail.com                                2023-09-30 00:00:00+00:00         1   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00         1   
104843199+jlcoulter@users.noreply.gith

In [21]:
df_gr = df_x.groupby(['author.email', pd.Grouper(key='author_date', freq=resample_rate)])
df_gr

In [22]:
df_gr[columns_agg].agg(
        agg_func_map,
        numeric_only=True
    )

n_commits  \
author.email                                    author_date                            
0002e7@gmail.com                                2023-09-30 00:00:00+00:00          1   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00          1   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00          2   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00          5   
                                                2024-09-30 00:00:00+00:00          3   
...                                                                              ...   
zordsdavini@gmail.com                           2020-06-30 00:00:00+00:00         12   
                                                2020-09-30 00:00:00+00:00          1   
                                                2021-03-31 00:00:00+00:00          1   
                                                2021-12-31 00:00:00+00:00          1   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00          1   

                                                                           +:count  \
author.email                                    author_date                          
0002e7@gmail.com                                2023-09-30 00:00:00+00:00     25.0   
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00    125.0   
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00      2.0   
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00     93.0   
                                                2024-09-30 00:00:00+00:00    563.0   
...                                                                            ...   
zordsdavini@gmail.com                           2020-06-30 00:00:00+00:00    715.0   
                                                2020-09-30 00:00:00+00:00     31.0   
                                                2021-03-31 00:00:00+00:00      1.0   
                                                2021-12-31 00:00:00+00:00     14.0   
zwork101@gmail.com                              2024-06-30 00:00:00+00:00      2.0   

                                                                           -:count  
author.email                                    author_date                         
0002e7@gmail.com                                2023-09-30 00:00:00+00:00     13.0  
1024369+holocronweaver@users.noreply.github.com 2024-12-31 00:00:00+00:00      2.0  
104843199+jlcoulter@users.noreply.github.com    2022-09-30 00:00:00+00:00      2.0  
107062289+shyguyCreate@users.noreply.github.com 2024-06-30 00:00:00+00:00     41.0  
                                                2024-09-30 00:00:00+00:00     25.0  
...                                                                            ...  
zordsdavini@gmail.com                           2020-06-30 00:00:00+00:00     95.0  
                                                2020-09-30 00:00:00+00:00     22.0  
                                                2021-03-31 00:00:00+00:00      1.0  
                                                2021-12-31 00:00:00+00:00      0.0  
zwork101@gmail.com                              2024-06-30 00:00:00+00:00      1.0  

[791 rows x 3 columns]